In [1]:
import time
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import xlrd
import geopandas as gpd
import ee
import geemap

In [3]:
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

In [4]:
iac = gpd.read_file("D:\\Radiance Khushi\\Shape Files\\India\\India_AC.shp")
iac

,OBJECTID,ST_CODE,ST_NAME,DT_CODE,DIST_NAME,AC_NO,AC_NAME,PC_NO,PC_NAME,PC_ID,STATUS,Shape_Leng,Shape_Area,geometry
0,1,13,NAGALAND,1.0,MON,41,Tizit,1,NAGALAND,1301,Pre delimitation,1.381854,0.055845,"POLYGON ((94.94575 26.93518, 94.95510 26.93975..."
1,1,13,NAGALAND,1.0,MON,43,Tapi,1,NAGALAND,1301,Pre delimitation,1.056157,0.030387,"POLYGON ((95.22324 26.75964, 95.21760 26.75589..."
2,1,13,NAGALAND,1.0,MON,42,Wakching,1,NAGALAND,1301,Pre delimitation,0.980303,0.018828,"POLYGON ((94.86775 26.82831, 94.87219 26.82334..."
3,1,13,NAGALAND,2.0,TUENSANG,49,Tamlu,1,NAGALAND,1301,Pre delimitation,1.133296,0.021899,"POLYGON ((94.73863 26.76868, 94.74029 26.77594..."
4,1,13,NAGALAND,3.0,MOKOKCHUNG,21,Tuli,1,NAGALAND,1301,Pre delimitation,0.965989,0.022397,"POLYGON ((94.73863 26.76868, 94.73627 26.74956..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4177,80,9,UTTAR PRADESH,70.0,SONBHADRA,400,Ghorawal,80,ROBERTSGANJ (SC),980,NaN,2.354558,0.104874,"POLYGON ((83.09302 24.79782, 83.08635 24.78159..."
4178,80,9,UTTAR PRADESH,70.0,SONBHADRA,401,Robertsganj,80,ROBERTSGANJ (SC),980,NaN,2.316555,0.161227,"POLYGON ((83.39350 24.78226, 83.42807 24.76825..."
4179,80,9,UTTAR PRADESH,70.0,SONBHADRA,402,Obra,80,ROBERTSGANJ (SC),980,NaN,2.437449,0.135870,"POLYGON ((83.09499 24.65526, 83.09988 24.65671..."
4180,80,9,UTTAR PRADESH,70.0,SONBHADRA,403,Duddhi (SC),80,ROBERTSGANJ (SC),980,NaN,2.631747,0.202402,"POLYGON ((83.27369 24.36497, 83.28408 24.35337..."


In [5]:
iacdf=pd.DataFrame(iac)

In [6]:
iacdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4182 entries, 0 to 4181
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID    4182 non-null   int64   
 1   ST_CODE     4182 non-null   int64   
 2   ST_NAME     4182 non-null   object  
 3   DT_CODE     4112 non-null   float64 
 4   DIST_NAME   4110 non-null   object  
 5   AC_NO       4182 non-null   int64   
 6   AC_NAME     4148 non-null   object  
 7   PC_NO       4182 non-null   int64   
 8   PC_NAME     4148 non-null   object  
 9   PC_ID       4182 non-null   int64   
 10  STATUS      524 non-null    object  
 11  Shape_Leng  4182 non-null   float64 
 12  Shape_Area  4182 non-null   float64 
 13  geometry    4182 non-null   geometry
dtypes: float64(3), geometry(1), int64(5), object(5)
memory usage: 457.5+ KB


In [7]:
rows=iacdf.shape[0]
cols=iacdf.shape[1]
print('Rows:',rows)
print('Columns:',cols)

Rows: 4182
Columns: 14


In [8]:
#Naming the AC's with unique names whoes names are None

for i in range(0,rows):
    if(pd.isna(iacdf.iloc[i,6])):
        x='a'+str(i)
        iacdf.iloc[i,6]=x

In [9]:
unames=list(iac['AC_NAME'].unique())

In [10]:
#Appending all the unique AC's to a dictionary
acnamedict={}
for i in unames:
    acnamedict[i]=0
    
#Iterating over AC's to name them uniquely
for i in range(rows):
    acname=iacdf.iloc[i,6]
    if acnamedict[acname]==0:
        acnamedict[acname]=1
    else:
        k=acnamedict[acname]
        iacdf.iloc[i,6]=iacdf.iloc[i,6]+str(k)
        acnamedict[acname]+=1

In [11]:
#Appending all the AC names into a dictionary
data={}
for i in iacdf['AC_NAME']:
    data[i]=[]

In [12]:
gdf = gpd.GeoDataFrame(iacdf)

In [ ]:
#number of days in each month into a list
j=[31,30,30,31]
m=[1,4,7,10]
# Dates in month
for i in range(4):
    #importing the images and filtering according to date by taking average radiation per month
    viirs2019_12=ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate("2019-{}-01".format(m[i]),"2019-{}-{}".format(m[i]+2,j[i])).select('avg_rad').mean()
    for k in range(rows):
        if (k%500==0):
            time.sleep(random.randint(0,30))
        acname=iacdf.iloc[k,6]
        try:
            ee_object2 = geemap.shp_to_ee(r'D:\Radiance Khushi\Shape Files\Constituencies\{}.shp'.format(acname)) #using the created shape file of the AC to get the object of Earth engine type
            viirs2019_12_clipped = viirs2019_12.clip(ee_object2) # c1
            affine = [0.00026949458523585647, 0, -180, 0, -0.00026949458523585647, 86.0000269494563]#parameters of matrix for 3D transformation
            #clips are reduced to the mean of the region using the function
            mu = viirs2019_12_clipped.reduceRegion(reducer=ee.Reducer.mean(), geometry = ee_object2.geometry(), crs = 'EPSG:4326', crsTransform = affine, maxPixels = 1e9)
            mu = ee.Number(mu.get('avg_rad'))
            print('Q'+str(i+1),k,acname)
            #average radiation per month by region
            print(f"The mean avg radiance (per month) for the pixels is: {mu.getInfo()}")
            #appending them to a list
            data[acname].append(mu.getInfo())
        except:
            data[acname].append(0.0)

Q1 0 Tizit
The mean avg radiance (per month) for the pixels is: 0.28823528432858997
Q1 1 Tapi
The mean avg radiance (per month) for the pixels is: 0.23225317463544295
Q1 2 Wakching
The mean avg radiance (per month) for the pixels is: 0.3093884949604919
Q1 3 Tamlu
The mean avg radiance (per month) for the pixels is: 0.2786691721573472
Q1 4 Tuli
The mean avg radiance (per month) for the pixels is: 0.3306874339788851
Q1 5 Phomching
The mean avg radiance (per month) for the pixels is: 0.16393462890165791
Q1 6 Mon Town
The mean avg radiance (per month) for the pixels is: 0.3453809424441508
Q1 7 Aboi
The mean avg radiance (per month) for the pixels is: 0.1987599325871628
Q1 8 Alungtaki
The mean avg radiance (per month) for the pixels is: 0.28211292065051563
Q1 9 Tehok
The mean avg radiance (per month) for the pixels is: 0.19602793088011994
Q1 10 Longleng
The mean avg radiance (per month) for the pixels is: 0.25591692795412896
Q1 11 Arkakong
The mean avg radiance (per month) for the pixels is

Q1 94 Dhekiajuli
The mean avg radiance (per month) for the pixels is: 0.5817130480694773
Q1 95 Sootea
The mean avg radiance (per month) for the pixels is: 0.4721471530620008
Q1 96 Majbat
The mean avg radiance (per month) for the pixels is: 0.7904618658884609
Q1 97 Biswanath
The mean avg radiance (per month) for the pixels is: 0.5123732163542655
Q1 98 Udalguri
The mean avg radiance (per month) for the pixels is: 0.7282439053556504
Q1 99 Panery
The mean avg radiance (per month) for the pixels is: 0.6064903042948875
Q1 100 Khumtai
The mean avg radiance (per month) for the pixels is: 0.9286179586948421
Q1 101 Titabar
The mean avg radiance (per month) for the pixels is: 0.6733437900482908
Q1 102 Barchalla
The mean avg radiance (per month) for the pixels is: 0.46951528903158257
Q1 103 Sidli
The mean avg radiance (per month) for the pixels is: 0.4020469316033351
Q1 104 Barama
The mean avg radiance (per month) for the pixels is: 0.592954482710186
Q1 105 Tamulpur
The mean avg radiance (per mont

Q1 187 Sonai
The mean avg radiance (per month) for the pixels is: 0.8811954098862507
Q1 188 Dholai
The mean avg radiance (per month) for the pixels is: 0.2614047412469835
Q1 189 Ratabari
The mean avg radiance (per month) for the pixels is: 0.42043143136680355
Q1 190 Patharkandi
The mean avg radiance (per month) for the pixels is: 0.5377667071554627
Q1 191 Hailakandi
The mean avg radiance (per month) for the pixels is: 0.7778935669862408
Q1 192 Katlicherra
The mean avg radiance (per month) for the pixels is: 0.33004622605458545
Q1 193 a193
The mean avg radiance (per month) for the pixels is: 0.3655564507374121
Q1 194 a194
The mean avg radiance (per month) for the pixels is: 0.2834047027289056
Q1 195 NURBRA
The mean avg radiance (per month) for the pixels is: 0.31513702483894274
Q1 196 KARGIL
The mean avg radiance (per month) for the pixels is: 0.3369752462619299
Q1 197 KARNAH
The mean avg radiance (per month) for the pixels is: 0.37514301958077445
Q1 198 LOLAB
The mean avg radiance (per

Q1 281 BANI
The mean avg radiance (per month) for the pixels is: 0.3061459433984103
Q1 282 JAMMU WEST
The mean avg radiance (per month) for the pixels is: 19.86710844973256
Q1 283 JAMMU EAST
The mean avg radiance (per month) for the pixels is: 20.568079139941663
Q1 284 SAMBA
The mean avg radiance (per month) for the pixels is: 1.5941837280870892
Q1 285 VIJAYPUR
The mean avg radiance (per month) for the pixels is: 0.289687641405503
Q1 286 GANDHI NAGAR
The mean avg radiance (per month) for the pixels is: 35.8065333986519
Q1 287 RAM NAGAR1
The mean avg radiance (per month) for the pixels is: 0.639251286387281
Q1 288 SAMBA1
The mean avg radiance (per month) for the pixels is: 10.121839689930285
Q1 289 VIJAYPUR1
The mean avg radiance (per month) for the pixels is: 3.7349674522736236
Q1 290 RAM NAGAR2
The mean avg radiance (per month) for the pixels is: 0.6078180514259948
Q1 291 BILLAWAR
The mean avg radiance (per month) for the pixels is: 0.8415221374874604
Q1 292 SUCHET GARH
The mean avg r

Q1 373 Kodungallur
The mean avg radiance (per month) for the pixels is: 1.2358215985502645
Q1 374 Angamaly
The mean avg radiance (per month) for the pixels is: 1.0091243981613374
Q1 375 Kothamangalam
The mean avg radiance (per month) for the pixels is: 0.1597677317796051
Q1 376 Paravur
The mean avg radiance (per month) for the pixels is: 1.757503960526742
Q1 377 Vypeen
The mean avg radiance (per month) for the pixels is: 2.8772040137198847
Q1 378 Aluva
The mean avg radiance (per month) for the pixels is: 4.583416114012854
Q1 379 Perumbavoor
The mean avg radiance (per month) for the pixels is: 1.2646366849005835
Q1 380 Kalamassery
The mean avg radiance (per month) for the pixels is: 5.166283496133081
Q1 381 Kothamangalam1
The mean avg radiance (per month) for the pixels is: 0.6501476593255285
Q1 382 Kunnathunad (SC)
The mean avg radiance (per month) for the pixels is: 4.442974479850711
Q1 383 Ernakulam
The mean avg radiance (per month) for the pixels is: 14.681397916389185
Q1 384 Muvatt

...

The mean avg radiance (per month) for the pixels is: 0.3666176397796491
Q4 3740 Bhagwanpur (SC)
The mean avg radiance (per month) for the pixels is: 2.4996272890207867
Q4 3741 Bageshwar (SC)
The mean avg radiance (per month) for the pixels is: 0.4755445840531202
Q4 3742 Hardwar
The mean avg radiance (per month) for the pixels is: 15.33374781806234
Q4 3743 Pirankaliyar
The mean avg radiance (per month) for the pixels is: 3.7356562724919637
Q4 3744 Hardwar Rural
The mean avg radiance (per month) for the pixels is: 1.664306347877861
Q4 3745 Salt
The mean avg radiance (per month) for the pixels is: 0.3727549601473113
Q4 3746 Dwarahat
The mean avg radiance (per month) for the pixels is: 0.3852884355694556
Q4 3747 Jhabrera (SC)
The mean avg radiance (per month) for the pixels is: 3.0745816285921825
Q4 3748 Roorkee
The mean avg radiance (per month) for the pixels is: 12.7714668507537
Q4 3749 Lansdowne
The mean avg radiance (per month) for the pixels is: 0.3125354731931451
Q4 3750 Khanpur1
The

Q4 3831 Hasanpur1
The mean avg radiance (per month) for the pixels is: 1.589034908812513
Q4 3832 Sambhal
The mean avg radiance (per month) for the pixels is: 2.0940058588560544
Q4 3833 Hapur (SC)
The mean avg radiance (per month) for the pixels is: 3.850217920578629
Q4 3834 Barkhera
The mean avg radiance (per month) for the pixels is: 0.89096636603429
Q4 3835 Puranpur (SC)
The mean avg radiance (per month) for the pixels is: 0.5086501402589123
Q4 3836 Dholana
The mean avg radiance (per month) for the pixels is: 7.52445041742354
Q4 3837 Meerganj
The mean avg radiance (per month) for the pixels is: 1.178723973735908
Q4 3838 Bhojipura
The mean avg radiance (per month) for the pixels is: 2.41550884903823
Q4 3839 Ghaziabad
The mean avg radiance (per month) for the pixels is: 30.37043897333546
Q4 3840 Syana
The mean avg radiance (per month) for the pixels is: 1.2566513751497688
Q4 3841 Sahibabad
The mean avg radiance (per month) for the pixels is: 27.669943236716108
Q4 3842 Milak (SC)
The me

Q4 3923 Mathura
The mean avg radiance (per month) for the pixels is: 10.117934170631703
Q4 3924 Hardoi
The mean avg radiance (per month) for the pixels is: 1.2126109099992193
Q4 3925 Balrampur (SC)
The mean avg radiance (per month) for the pixels is: 1.2306675008495191
Q4 3926 Jasrana
The mean avg radiance (per month) for the pixels is: 1.1105533415970068
Q4 3927 Payagpur
The mean avg radiance (per month) for the pixels is: 0.9807216207801698
Q4 3928 Misrikh (SC)
The mean avg radiance (per month) for the pixels is: 1.042679596197767
Q4 3929 Mahmoodabad
The mean avg radiance (per month) for the pixels is: 0.7436034233180738
Q4 3930 Shohratgarh
The mean avg radiance (per month) for the pixels is: 0.7150645373059566
Q4 3931 Mainpuri
The mean avg radiance (per month) for the pixels is: 1.8643251633551838
Q4 3932 Kaiserganj
The mean avg radiance (per month) for the pixels is: 0.8802103218429512
Q4 3933 Nautanwa
The mean avg radiance (per month) for the pixels is: 0.8383853553196206
Q4 3934 

Q4 4014 Lucknow Cantt.
The mean avg radiance (per month) for the pixels is: 30.020307880508327
Q4 4015 Mohanlalganj (SC)
The mean avg radiance (per month) for the pixels is: 1.985919142981351
Q4 4016 Rasulabad (SC)
The mean avg radiance (per month) for the pixels is: 0.9148702420822229
Q4 4017 Bikapur
The mean avg radiance (per month) for the pixels is: 1.4832542047769968
Q4 4018 Ayodhya
The mean avg radiance (per month) for the pixels is: 3.7561017922900355
Q4 4019 Dibiyapur
The mean avg radiance (per month) for the pixels is: 2.8274539063133477
Q4 4020 Gorakhpur Rural
The mean avg radiance (per month) for the pixels is: 4.716816605096268
Q4 4021 Haidergarh (SC)
The mean avg radiance (per month) for the pixels is: 1.2610296176679283
Q4 4022 Pathardeva
The mean avg radiance (per month) for the pixels is: 1.1018936195823936
Q4 4023 Chauri-Chaura
The mean avg radiance (per month) for the pixels is: 1.9236017751419763
Q4 4024 Mahadewa (SC)
The mean avg radiance (per month) for the pixels 

Q4 4105 Mehnagar (SC)
The mean avg radiance (per month) for the pixels is: 1.274066434372782
Q4 4106 Bairia
The mean avg radiance (per month) for the pixels is: 1.5033732253908716
Q4 4107 Lalganj (SC)
The mean avg radiance (per month) for the pixels is: 1.3355191972918667
Q4 4108 Raniganj
The mean avg radiance (per month) for the pixels is: 1.3740982792818794
Q4 4109 Rath (SC)
The mean avg radiance (per month) for the pixels is: 0.561081104301095
Q4 4110 Babaganj (SC)
The mean avg radiance (per month) for the pixels is: 0.8957943475542014
Q4 4111 Ayah Shah
The mean avg radiance (per month) for the pixels is: 0.7533671530559777
Q4 4112 Phephana
The mean avg radiance (per month) for the pixels is: 1.4570702699571785
Q4 4113 Tindwari
The mean avg radiance (per month) for the pixels is: 0.7791043504019416
Q4 4114 Malhani
The mean avg radiance (per month) for the pixels is: 2.519753052057501
Q4 4115 Zahoorabad
The mean avg radiance (per month) for the pixels is: 1.5853865510376322
Q4 4116 J

In [ ]:
meanrad2019=pd.DataFrame()
raddata=list(data.items())
findict = {raddata[i][0]: raddata[i][1] for i in range(0, len(raddata))}
namecol=[]
rowdata=[[] for i in range(4)]

for key,value in findict.items():
    namecol.append(key)
    for j in range(0,4):
        rowdata[j].append(value[j])
        
meanrad2019['AC_NAME']=namecol
quart=['Quarter 1','Quarter 2','Quarter 3','Quarter 4']
for i in range(4):
     meanrad2019[quart[i]+"-2019"]=rowdata[i]
        
meanrad2019

In [ ]:
meanrad2019.to_csv("radiation2019.csv",index=False)